In [15]:
#스크리닝 전체 코드
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import json
import requests

from pykrx import stock

data = stock.get_market_cap("20240219", market="ALL") #3개월의 투자 수익률 확인 위해
data = data[['종가', '시가총액']]
df = stock.get_market_fundamental_by_ticker("20240219", market='ALL') 
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']] 
df3 = pd.merge(data, df2, on='티커')

df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]

df = stock.get_market_fundamental("20240104", "20240214", '006840', freq="m")
df = df.T

ticker = df3.index
eps_change_data = []

for stock_code in ticker:
    df = stock.get_market_fundamental("20240104", "20240214", stock_code, freq="m")
    df = df.T
    df['변화율'] = (df.iloc[:, 1] - df.iloc[:, 0]) / df.iloc[:, 0]
    eps_change_data.append(df['변화율'])

# DataFrame 생성
eps_change_df = pd.DataFrame(eps_change_data, index=ticker)

mean = eps_change_df['EPS'].mean()
eps_change_df = eps_change_df.loc[eps_change_df['EPS'] <= mean]
ticker = eps_change_df.index.to_list()

df = stock.get_market_ohlcv("20240119", "20240219", '024110', "m") #직전 한달 주가 수익률
gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100

tickers = eps_change_df.index.to_list()
gains = []

for ticker in tickers:
    df = stock.get_market_ohlcv("20240119", "20240219", ticker, "m")
    gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
    gains.append(gain)

result_df = pd.DataFrame({'ticker': tickers, 'gain': gains})
result_df.set_index('ticker', inplace=True)

gain_25 = result_df['gain'].quantile(0.25)
selected_rows = result_df['gain'] < gain_25

temp_df = result_df[selected_rows]
temp_df.index.nunique() #36개의 종목 필터링

tickers = temp_df.index.tolist()
revisions = []

for ticker in tickers:
    data = stock.get_market_ohlcv("20240119", "20240219", ticker, "d")
    mean = data['종가'].mean()
    close_temp = stock.get_market_ohlcv("20240119", "20240219", ticker, "d")
    close = close_temp.iloc[0,3]
    revision = (close / mean) * 100
    revisions.append(revision)

# 새로운 데이터프레임 생성
result_df = pd.DataFrame({'ticker': tickers, 'revision': revisions})
result_df.set_index('ticker', inplace=True)
revision_50 = result_df['revision'].quantile(0.5)
selected_rows = result_df['revision'] < revision_50

revision_df = result_df[selected_rows]

tickers = revision_df.index
mean_values = []

for ticker in tickers:
    data1 = stock.get_market_trading_value_by_date("20240119", "20240219", ticker)
    data2 = stock.get_market_cap("20240119", "20240219", ticker)['시가총액']
    data1['개인순매수'] = (data1['개인'] / data2) * 100
    mean = data1['개인순매수'].mean()
    mean_values.append(mean)

final_df = pd.DataFrame({'ticker': tickers, 'mean': mean_values})
final_df.set_index('ticker', inplace=True)

final_50 = final_df['mean'].quantile(0.5)
selected_rows = final_df['mean'] < final_50
screening = final_df[selected_rows]
screening.index

Index(['139480', '375500', '001120', '003240', '000670', '007690', '035510',
       '000950'],
      dtype='object', name='ticker')

In [191]:
import pandas as pd
import numpy as np
from pykrx import stock

def perform_screening(end_date):
    end_date_datetime = datetime.strptime(end_date, "%Y%m%d")
    start_date_datetime = end_date_datetime - timedelta(days=30)
    start_date = start_date_datetime.strftime("%Y%m%d")

    # 시가총액 데이터 가져오기
    market_cap_data = stock.get_market_cap(end_date, market="ALL")
    market_cap_data = market_cap_data[['종가', '시가총액']]

    # 기본적인 재무제표 데이터 가져오기
    fundamental_data = stock.get_market_fundamental_by_ticker(end_date, market='ALL')
    column_pbr = fundamental_data['PBR'].values
    pbr_25 = np.percentile(column_pbr, 25)
    filtered_fundamental_data = fundamental_data.loc[(fundamental_data['PER'] < 5) & (fundamental_data['PBR'] < pbr_25)]
    filtered_fundamental_data = filtered_fundamental_data[['PER', 'PBR']]

    # 종목별 시가총액과 필터링된 재무제표 데이터 병합
    merged_data = pd.merge(market_cap_data, filtered_fundamental_data, on='티커')

    # 재무제표 필터링
    merged_data = merged_data.loc[(merged_data['PER'] > 0) & (merged_data['PBR'] > 0)]
    merged_data = merged_data[['종가', 'PER', 'PBR']]

    # EPS 변화율 계산 및 필터링
    ticker_list = merged_data.index.to_list()
    eps_change_data = []

    for stock_code in ticker_list:
        df = stock.get_market_fundamental(start_date, end_date, stock_code, freq="m").T
        df['변화율'] = (df.iloc[:, 1] - df.iloc[:, 0]) / df.iloc[:, 0]
        eps_change_data.append(df['변화율'])

    eps_change_df = pd.DataFrame(eps_change_data, index=ticker_list)
    mean_eps = eps_change_df['EPS'].mean()
    eps_change_df = eps_change_df.loc[eps_change_df['EPS'] <= mean_eps]
    filtered_tickers = eps_change_df.index.to_list()

    # 한달 주가 수익률 계산 및 필터링
    gains = []

    for ticker in filtered_tickers:
        df = stock.get_market_ohlcv(start_date, end_date, ticker, "m")
        gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
        gains.append(gain)

    result_df = pd.DataFrame({'ticker': filtered_tickers, 'gain': gains})
    result_df.set_index('ticker', inplace=True)
    gain_25 = result_df['gain'].quantile(0.25)
    selected_rows = result_df['gain'] < gain_25
    temp_df = result_df[selected_rows]
    selected_tickers = temp_df.index.tolist()

    # 평균값 계산
    revisions = []

    for ticker in selected_tickers:
        data = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        mean_price = data['종가'].mean()
        close_temp = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        close = close_temp.iloc[0,3]
        revision = (close / mean_price) * 100
        revisions.append(revision)

    result_df = pd.DataFrame({'ticker': selected_tickers, 'revision': revisions})
    result_df.set_index('ticker', inplace=True)
    revision_50 = result_df['revision'].quantile(0.5)
    selected_rows = result_df['revision'] < revision_50
    revision_df = result_df[selected_rows]

    # 평균값 계산
    mean_values = []

    for ticker in revision_df.index:
        data1 = stock.get_market_trading_value_by_date(start_date, end_date, ticker)
        data2 = stock.get_market_cap(start_date, end_date, ticker)['시가총액']
        data1['개인순매수'] = (data1['개인'] / data2) * 100
        mean = data1['개인순매수'].mean()
        mean_values.append(mean)

    final_df = pd.DataFrame({'ticker': revision_df.index, 'mean': mean_values})
    final_df.set_index('ticker', inplace=True)
    final_50 = final_df['mean'].quantile(0.5)
    selected_rows = final_df['mean'] < final_50
    screening = final_df[selected_rows]

    return screening

# 스크리닝 함수 호출
end_date = "20240322"
screened_stocks = perform_screening(end_date)
print(screened_stocks.index)

Index(['003380', '014280', '002460', '007980', '024800', '007680', '016250',
       '050860'],
      dtype='object', name='ticker')


In [237]:
# 백테스트 코드
# 간단하게 CAGR만 활용하였습니다
# 투자원금
cash = 100000000 # 모의투자 시작금액인 1억으로 시작하겠습니다.
money = cash

if len(screened_stocks.index)==0:
    allocation=0
else:
    allocation = money / len(screened_stocks.index) # 동일 비중이기 때문에 보유 현금 / 투자할 종목수 로 나눠줍니다.
    
final_df['매수 수량'] = allocation / final_df['종가']
final_df

,시가,종가,매수 수량
ticker,,,
139480,69300,69400,180.115274
375500,36100,36400,343.406593
001120,26100,26050,479.846449
003240,725000,717000,17.433752
007690,38350,38050,328.515112
035510,11950,12180,1026.272578
009410,0,2310,5411.255411
000950,22950,23150,539.956803


In [255]:
import FinanceDataReader as fdr #벤치마킹 지수

ref = fdr.DataReader(
    symbol = 'KS11',
    start = '20190319',
    end = '20240322')

num_of_year = int(len(ref.index)/365) #90일을 잡았습니다
ref_cagr = ((ref['Close'].iloc[-1] / ref['Close'].iloc[0])**(1/num_of_year)) -1
ref_cagr*100

ref['agr'] = (ref['Close'] / ref['Open'])
ref['cagr'] = ((ref['Close'] / ref['Open']))**(1/num_of_year) -1

tickers = screened_stocks.index
date_range = pd.date_range(start="20190319", end="20240322")
result_df = pd.DataFrame(index=date_range)

for ticker in tickers:
    data = stock.get_market_ohlcv("20190319", "20240322", ticker, "d")
    data['수익률'] = (data['종가'] / data['시가'])
    result_df[ticker] = data['수익률']

# NaN 값을 이전 행의 값으로 채우기
result_df.fillna(method='ffill', inplace=True)

result_df.index.name = '날짜'

# DataFrame에서 무한대 값을 NaN으로 대체
result_df.replace([np.inf, -np.inf], np.nan, inplace=True)
result_df['포트폴리오 수익률'] = result_df.mean(axis=1, skipna=True) 
result_df['CAGR'] = (data['수익률'] ** (1 / num_of_year) - 1) * 100
result_df

,055550,137310,000670,035890,004890,101330,091590,000950,106240,포트폴리오 수익률,CAGR
날짜,,,,,,,,,,,
2019-03-19,0.989865,NaN,0.987283,0.995902,1.000000,1.012146,1.000000,1.008287,1.028090,1.002697,0.927697
2019-03-20,1.006849,NaN,1.007075,0.987654,1.000000,0.997000,0.996819,0.989041,1.074278,1.007340,2.417037
2019-03-21,0.987457,NaN,0.987059,0.991632,0.993174,1.008105,0.990426,0.991690,0.923175,0.984090,-2.629348
2019-03-22,1.017647,NaN,0.997590,1.000000,1.013746,1.061753,0.992400,1.013889,1.013966,1.013874,0.463399
2019-03-23,1.017647,NaN,0.997590,1.000000,1.013746,1.061753,0.992400,1.013889,1.013966,1.013874,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-18,0.987891,0.982699,0.973113,1.000000,0.996587,0.994286,0.971377,1.015695,1.011463,0.992568,0.380657
2024-03-19,0.993769,0.974359,0.995745,0.999225,1.004566,0.981295,1.000000,0.980435,0.990667,0.991118,-0.312084
2024-03-20,1.004115,1.041108,0.986258,0.997674,1.000000,1.010264,0.988785,1.026608,0.989233,1.004894,-0.360201


In [257]:
mean_return = result_df['포트폴리오 수익률'].mean()
(mean_return - 1) * 100

0.06524122966538037

In [239]:
import plotly.graph_objs as go
import plotly.io as pio

pio.templates["seaborn"] = go.layout.Template(layout=go.Layout(template="seaborn"))
pio.templates.default = "seaborn"

scatter_plot = go.Scatter(x=ref.index, y=ref['cagr'], mode='lines', name='코스피 CAGR')
scatter_plot_cagr = go.Scatter(x=result_df.index, y=result_df['CAGR'], mode='lines', name='포트폴리오 CAGR')

layout = go.Layout(title='<b>5년 백테스팅<b>',
                   xaxis=dict(title='날짜'),
                   yaxis=dict(title='CAGR'))

# 그래프 객체를 figure로 생성합니다.
fig = go.Figure(data=[scatter_plot, scatter_plot_cagr], layout=layout)

# 그래프 출력
fig.show()

In [240]:
import plotly.graph_objs as go
import plotly.io as pio

pio.templates["seaborn"] = go.layout.Template(layout=go.Layout(template="seaborn"))
pio.templates.default = "seaborn"

scatter_plot = go.Scatter(x=ref.index, y=ref['agr'], mode='lines', name='코스피 수익률')
scatter_plot_mean = go.Scatter(x=result_df.index, y=result_df['포트폴리오 수익률'], mode='lines', name='포트폴리오 수익률')

layout = go.Layout(title='<b>5년 백테스팅<b>',
                   xaxis=dict(title='날짜'),
                   yaxis=dict(title='수익률'))

# 그래프 객체를 figure로 생성합니다.
fig = go.Figure(data=[scatter_plot, scatter_plot_mean], layout=layout)

# 그래프 출력
fig.show()